#Transitive Win Program

The purpose of this notebook is to create a method for ranking College Football teams in a given year based on how many total teams each individual team should be able to beat based on the Transitive Property. While not perfect, this method can be helpful in comparing teams this sport as it can be difficult as there are 130 teams playing at the highest level and each team only plays about 12/13 games a year.

## Functions



In [ ]:
# Installing and importing necessary functions and modules
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install beautifulsoup4 as bs4

from lxml import html
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import re 
import types
import pandas as pd
from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


This function pulls data from Wikipedia to determine the teams that are classified into the highest "FBS" level of College Football for a given year.


In [ ]:
def get_Teams(input_year):
    if 1941 <= input_year <= 1945:
        input_year = 1940
    url="https://en.wikipedia.org/wiki/NCAA_Division_I_Football_Bowl_Subdivision_alignment_history"
    # use urllib and BeautifulSoup in order to open and copy the html code of the webpage to another variable
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
    # use soup.find in order to pull out only the html code related to the data wanted
    team_list = soup.findAll('div')
    A = []
    for row in soup.findAll('li'):
        strip = row.text.strip()
        A.append(strip)
    team_list
    A = A[26:-50]

    B = []
    C = []
    D = []
    stop = "1"
    for s in A:
        index = s.find(" 1")
        index2 = s.find(" 2")
        if index2 == -1 or index < index2 and index != -1:
            B.append(s[:index])
        else:
            B.append(s[:index2])

    df = pd.DataFrame(A,columns = ['Team1'])
    df['Team'] = B

    df.replace("as West Texas State","", inplace=True, regex = True)
    df.replace(" \(\)","", inplace=True, regex = True)
    df.replace("present","2021", inplace=True, regex = True) # update each year
    df.replace("future","2024", inplace=True, regex = True) # update each year
    df.replace("1981","AAAA", inplace = True, regex=True)
    df.replace("81","1981", inplace = True, regex=True)
    df.replace("AAAA","1981", inplace = True, regex=True)
    df.replace("1940","AAAA", inplace = True, regex=True)
    df.replace("40","1940", inplace = True, regex=True)
    df.replace("AAAA","1940", inplace = True, regex=True)
    df.replace("-40","-1981", inplace = True, regex=True)
    df.replace("Southern California","USC", inplace = True, regex=True)


    for s, s2 in zip(df['Team1'], df['Team']):
        s3 = s.replace(s2,"")
        C.append(s3)

    np.linspace(1940,1946,7)
    for s in C:
        a = []
        length = len(s)
        char = 1
        while char < length:
            if length < 6:
                a.append(int(s[char:char+4]))
                break
            if s == " 2022–2021[a]":
                a.append(2022)
                break
            if s[char+4] == ',':
                a.append(int(s[char:char+4]))
                char = char + 6
            else:
                year1 = int(s[char:char+4])
                year2 = int(s[char+5:char+9])
                years = range(year1, year2+1)
                for year in years:
                    a.append(year)
                char = char + 11
        D.append(a)
           
    df['Years'] = D
    df.drop(['Team1'], axis = 1, inplace = True)
    df
    Team_List = []
    if input_year >= 1937:
        for team, years in zip(df['Team'], df['Years']):
            if input_year in years:
                Team_List.append(team)
    if input_year < 1937:
        Team_List = df['Team'].to_list()
    return(Team_List)

This function pulls all the College Football games from sports-reference.com for a given year.

In [ ]:
def get_Games(year):

    url="https://www.sports-reference.com/cfb/years/{}-schedule.html".format(year)
# use urllib and BeautifulSoup in order to open and copy the html code of the webpage to another variable
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
# use soup.find in order to pull out only the html code related to the data wanted
    game_list = soup.findAll('tr')
#strip = game_list.text.strip()
#print(game_list.findAll('tr'))

    A = []
    B = []
    C = []
    D = []
    E = []
    F = []
    G = []
    H = []
    I = []
    J = []

    for row in game_list:
        cells = row.findAll('td')
        if len(cells) == 10:
            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            C.append(cells[2].find(text=True))
            D.append(cells[3].find(text=True))
            E_find=str(cells[4].find('a'))
            start = E_find.find(">")
            end = E_find.find('<',start)
            if E_find == "None":
                E.append(cells[4].find(text=True))
            else:
                E.append(str(E_find[start+1:end])) 
            F.append(cells[5].find(text=True))
            G.append(cells[6].find(text=True))
            H_find=str(cells[7].find('a'))
            start = H_find.find(">")
            end = H_find.find('<',start)
            if H_find == "None":
                H.append(cells[7].find(text=True))
            else:
                H.append(str(H_find[start+1:end]))      
            I.append(cells[8].find(text=True))
            J.append(cells[9].find(text=True))
        if len(cells) == 9:
            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            D.append(cells[2].find(text=True))
            E_find=str(cells[3].find('a'))
            start = E_find.find(">")
            end = E_find.find('<',start)
            if E_find == "None":
                E.append(cells[3].find(text=True))
            else:
                E.append(str(E_find[start+1:end]))
            F.append(cells[4].find(text=True))
            G.append(cells[5].find(text=True))
            H_find=str(cells[6].find('a'))
            start = H_find.find(">")
            end = H_find.find('<',start)
            if H_find == "None":
                H.append(cells[6].find(text=True))
            else:
                H.append(str(H_find[start+1:end]))       
            I.append(cells[7].find(text=True))
            J.append(cells[8].find(text=True))
    df_Games = pd.DataFrame(A,columns = ['Wk'])
    df_Games['Date'] = B
   # df_Games['Time'] = C
    df_Games['Day'] = D
    df_Games['Winner'] = E
    df_Games['Pts'] = F
    df_Games['@'] = G
    df_Games['Loser'] = H
    df_Games['Pts.1'] = I
    #df_Games['Notes'] = J

#df_Games.drop(["Notes","Day","Date","Time","Unnamed: 7","Pts","Pts.1"], axis = 1, inplace = True)
#df_Games = pd.read_csv("2020 Games.csv")
# clean data by removing unneccessary extra characters
    df_Games["Winner"] = df_Games.Winner.str.replace('(\(\d\))','')   # ('(\(\d\))
    df_Games["Winner"] = df_Games.Winner.str.replace('(\(\d\d\))','')   # ('(\(\d\))
    df_Games["Winner"] = df_Games.Winner.str.replace('\xa0','')   # ('(\(\d\))
    df_Games["Loser"] = df_Games.Loser.str.replace('(\(\d\))','')   # ('(\(\d\))
    df_Games["Loser"] = df_Games.Loser.str.replace('(\(\d\d\))','')   # ('(\(\d\))
    df_Games["Loser"] = df_Games.Loser.str.replace('\xa0','')   # ('(\(\d\))

    df_Games.replace('Alabama-Birmingham','UAB', inplace = True)
    df_Games.replace('Southern Mississippi','Southern Miss', inplace = True)
    df_Games.replace('Middle Tennessee State','Middle Tennessee', inplace = True)
    df_Games.replace('Southern Methodist','SMU', inplace = True)
    df_Games.replace('Texas-El Paso','UTEP', inplace = True)
    df_Games.replace('Brigham Young','BYU', inplace = True)
    df_Games.replace('Louisiana-Monroe','Louisiana–Monroe', inplace = True)
    df_Games.replace('Texas-San Antonio','UTSA', inplace = True)
    df_Games.replace('Central Florida','UCF', inplace = True)
    df_Games.replace('Mississippi','Ole Miss', inplace = True)
    df_Games.replace('Texas Christian','TCU', inplace = True)
    df_Games.replace('Florida International','FIU', inplace = True)
    df_Games.replace('Louisiana State','LSU', inplace = True)
    df_Games.replace('Bowling Green State','Bowling Green', inplace = True)
    df_Games.replace('Texas A&amp;M','Texas A&M', inplace = True)
    df_Games.replace('Nevada-Las Vegas','UNLV', inplace = True)
    df_Games.replace('Connecticut','UConn', inplace = True)
    df_Games.replace("Saint Mary's (CA)","Saint Mary's", inplace = True)
    df_Games.replace('Virginia Military Institute','VMI', inplace = True)
    df_Games.replace('William &amp; Mary','William & Mary', inplace = True)
    df_Games.replace('Pennsylvania','Penn', inplace = True)
    df_Games.replace('West Texas A&amp;M','West Texas A&M', inplace = True)
    df_Games.replace('Washington &amp; Lee','Washington & Lee', inplace = True)
    df_Games.replace('Detroit Mercy','Detroit', inplace = True)
    df_Games.replace('Hardin-Simmons','Hardin–Simmons', inplace = True)
    df_Games.replace('New York University','NYU', inplace = True)
    df_Games.replace('Carnegie Mellon','Carnegie Tech', inplace = True)
    df_Games.replace('Texas-Arlington','Texas–Arlington', inplace = True)
    df_Games.replace('Southern','Southern U.', inplace = True)
    df_Games.replace('Grambling State','Grambling', inplace = True)
    df_Games.replace('Centenary (LA)','Centenary', inplace = True)
    df_Games.replace('Trinity (Texas)','Trinity (TX)', inplace = True)
    df_Games.replace('Southern California','USC', inplace = True)

    return (df_Games)

This function pulls the data for the final rankings for a given year.

In [ ]:
def get_Final_AP_Poll(year):
    if 1936 <= year <= 1955:
        url = "https://en.wikipedia.org/wiki/{}_NCAA_football_rankings".format(year)
    elif 1956 <= year <= 1972:
        url = "https://en.wikipedia.org/wiki/{}_NCAA_University_Division_football_rankings".format(year)
    elif 1973 <= year <= 1977:
        url = "https://en.wikipedia.org/wiki/{}_NCAA_Division_I_football_rankings".format(year)
    elif 1978 <= year <= 2005:
        url = "https://en.wikipedia.org/wiki/{}_NCAA_Division_I-A_football_rankings".format(year)
    elif 2006 <= year <= 2021:
        url = "https://en.wikipedia.org/wiki/{}_NCAA_Division_I_FBS_football_rankings".format(year)
    else:
        print("No AP Poll for this year")
    if year >= 1936:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'lxml')
        table = soup.find('table', class_='wikitable')
        A=[]
        B=[]
        C=[]
        D=[]
        E=[]
        F=[]
        i=0
        #print(table)
        for row in table.findAll('tr'):
            cells = row.findAll('td')
            if len(cells) != 0:
                #print(cells[-1])
                i=i+1
                #A.append(cells[-1].find(text=True))
                B.append(i)
                n = cells[-1]
                strip = n.text.strip()
                A.append(strip)
                #print(n[-10:])
                l = len(cells)
                A_find=str(cells[l-1].find('a'))
                #A.append(A_find)
        A.pop()
        B.pop()
        if year == 1940:
            A.pop()
            B.pop()
        if 2004 <= year <= 2006:
            A.pop()
            A.pop(0)
            B.pop()
            B.pop(0)
            G = []
            for num in B:
                num = num -1
                G.append(num)
            B = G
        B = []
        i = 0
        tie = 0
        for team in A:
            if team:
                i = i+1
                if tie%2 == 1:
                    B.append(i-1)
                else:
                    B.append(i)
                F.append(year)
                p = team.rfind("(")
                text = team[p-1:]
                if team.find('т') != -1:
                    tie = tie + 1
                team = team.replace(text,'')
                team = team.replace('т','')
                text = text.replace('т','')            
                check = text.find('–')
                if check == -1:
                    E.append(text)
                    p = team.rfind("(")
                    text = team[p-1:]
                    team = team.replace(text,'')
                    C.append(text)
                    D.append(team)
                else:
                    C.append(text)
                    D.append(team)
                    E.append('')
    if year < 1936:
        B = []
        C = []
        D = []
        E = []
        F = []
    df_Poll = pd.DataFrame(F,columns = ['Year'])
    df_Poll['Rank'] = B
    df_Poll['Team'] = D
    df_Poll['Record'] = C
    df_Poll['1st'] = E
    #df_Poll['year'] = F
    return df_Poll
    

This function looks through the df_Games dataframe to compile a list of all teams that played.

In [ ]:
def get_Teams2(df_Games):
    Teams2 = []
    for winner, loser in zip(df_Games['Winner'], df_Games['Loser']):
        if winner not in Teams2:
            Teams2.append(winner)
        if loser not in Teams2:
            Teams2.append(loser)
    return(Teams2)

This function cleans the df_Games dataframe to only include FBS vs FBS games.

In [ ]:
def clean_df(df_Games, Teams, Teams2, week):
    Diff = []
    for team in Teams2:
        if team not in Teams:
            Diff.append(team)
    for team in Teams:
        if team not in Teams2:
            Teams.remove(team)
    #print(len(Diff))
    df_Games['Wk'] = df_Games['Wk'].astype(int)
    check = []
    for winner, loser, points in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Wk']):
        if winner in Diff or loser in Diff or points > week:
            check.append('Remove')
        else:
            check.append('Keep')
    df_Games['check'] = check
    #print(df_Games.shape)
    df_Games = df_Games[df_Games['check'] == 'Keep']
#df_Games.drop(["Notes","Day","Date","Time","@","Pts","Pts.1","check"], axis = 1, inplace = True)
#df_Games.drop(['check'], axis = 1, inplace = True)
    #print(df_Games.shape)
    return(df_Games)
#df_Games

The next 3 functions find the teams that a given team has beat, lost to, or tied.

In [ ]:
def get_Wins(team, df_Games):
    Wins = []
    for winner, loser, P1, P2 in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Pts'], df_Games['Pts.1']):
        if winner == team and P1 != P2:
            Wins.append(loser)

    return (Wins)
#for winner, loser in zip(df_Games['Winner'], df_Games['Loser']):
 #   if winner in Teams:
  #      win_dict[winner].append(loser)
#win_dict

In [ ]:
def get_Losses(team, df_Games):
    Losses = []
    for winner, loser, P1, P2 in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Pts'], df_Games['Pts.1']):
        if loser == team and P1 != P2:
            Losses.append(winner)

    return (Losses)

In [ ]:
def get_Ties(team, df_Games):
    Ties = []
    for winner, loser, P1, P2 in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Pts'], df_Games['Pts.1']):
        if loser == team and P1 == P2:
            Ties.append(winner)
        if winner == team and P1 == P2:
            Ties.append(loser)
    return (Ties)

This function lists the teams remaining on a given team's schedule.

In [ ]:
def get_Remaining(team, df_Games, week, FBS):
    Remaining = []
    for winner, loser, w in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Wk']):
        if w > week:
            if winner == team:
                if loser in FBS:
                    Remaining.append(loser)
            if loser == team:
                if winner in FBS:
                    Remaining.append(winner)
    return (Remaining)

The next 2 functions use a teams wins/losses and those teams' wins/losses to find all transitive wins/losses within FBS.

In [ ]:
def get_T_wins(Team,df_Games):
    wins = get_Wins(Team,df_Games)
    control = []
    I = []
    for win in wins:
        if win not in control:
            control.append(win)
    win = control    
    T_wins = win
    teams_left = win# this list must not start empty  {}
    i = 1
    l = len(T_wins)
    I = [1]*l
    while teams_left:
        i=i+1
        difference = []
        for team in teams_left:
            #T_wins.append(team)
            #print(team)
            tlist = get_Wins(team,df_Games)
            for win in tlist:
                if win not in T_wins:
                    T_wins.append(win)
                    difference.append(win)
                    I.append(i)
        teams_left = difference 
    if Team in T_wins:
        T_wins.remove(Team)  
    return(T_wins)

In [ ]:
def get_T_losses(Team,df_Games):
    losses = get_Losses(Team,df_Games)
    control = []
    I = []
    for loss in losses:
        if loss not in control:
            control.append(loss)
    loss = control    
    T_losses = loss
    teams_left = loss# this list must not start empty  {}
    i = 1
    l = len(T_losses)
    I = [1]*l
    while teams_left:
        i=i+1
        difference = []
        for team in teams_left:
            #T_wins.append(team)
            #print(team)
            tlist = get_Losses(team,df_Games)
            for loss in tlist:
                if loss not in T_losses:
                    T_losses.append(loss)
                    difference.append(loss)
                    I.append(i)
        teams_left = difference 
    if Team in T_losses:
        T_losses.remove(Team)  
    return(T_losses)

This function determines if a team falls within a "cluster" defined in this program as a team with a transitive win over itself. Since in sports it is possible for an outcome such as: A>B B>C C>A, it is necessary to determine other factors to compare teams A, B, and C.

In [ ]:
def get_Cluster(Team,df_Games):
    wins = get_Wins(Team,df_Games)
    control = []
    I = []
    for win in wins:
        if win not in control:
            control.append(win)
    win = control    
    T_wins = wins
    teams_left = wins# this list must not start empty  {}
    i = 1
    l = len(T_wins)
    I = [1]*l
    while teams_left:
        i=i+1
        difference = []
        for team in teams_left:
            #T_wins.append(team)
            #print(team)
            tlist = get_Wins(team,df_Games)
            for win in tlist:
                if win not in T_wins:
                    T_wins.append(win)
                    difference.append(win)
                    I.append(i)
        teams_left = difference 
    if Team in T_wins:
        Cluster = True 
    else:
        Cluster = False
    return(Cluster)

In [ ]:
def get_df_T_Wins(Teams, df_Games):
    T_wins = []
    l = []
    m = []
    n = []
    p = []
    for team in Teams:
        t = (get_T_wins(team, df_Games))
        T_wins.append(t)
        l.append(len(t))

    df_T_wins = pd.DataFrame(Teams,columns = ['Team'])
    df_T_wins['#'] = l
    df_T_wins['Transitive Wins'] = T_wins 
    for wins, T in zip(T_wins, df_T_wins['Team']):
        n=[]
        for team in Teams:
            if team not in wins and team != T:
                n.append(team)
        m.append(n)
        p.append(len(n))
    df_T_wins['Missing'] = m
    #df_T_wins['#2'] = p
    #df_T_wins['check'] = df_T_wins['#'] +df_T_wins['#2'] 

    df_T_wins.sort_values(by= '#', ascending = False, inplace = True)
    return(df_T_wins)

In [ ]:
def get_Record(Team, df_Games):
    win = 0
    tie = 0
    loss = 0
    Record = []
    for winner, loser, pwin, ploss in zip(df_Games['Winner'],df_Games['Loser'],df_Games['Pts'],df_Games['Pts.1']):
        if Team == winner:
            if pwin != ploss:
                win = win+1
            if pwin == ploss and pwin != "None":
                tie = tie+1
        if Team == loser:
            if pwin != ploss:
                loss = loss+1
            if pwin == ploss and pwin != "None":
                tie = tie+1
    if tie == 0:
        record = "({}–{})".format(win,loss)
    if tie != 0:
        record = "({}–{}–{})".format(win,loss,tie)
     #   Record.append(record)
    #df_record = pd.DataFrame(Teams,columns = ['Team'])
    #df_record['Record'] = Record
    return(record)


In [ ]:
def get_Transitive_Wins(year):
    week = 30
    Teams = get_Teams(year)
    df_Games = get_Games(year)
    df_Games_All = df_Games.copy()
    Teams2 = get_Teams2(df_Games)
    Teams = [x for x in Teams2 if x not in Teams]
    Teams_Remove = Teams.copy()
    for team in Teams:
        if team not in Teams2:
            Teams_Remove.append(team)
    Teams = [x for x in Teams2 if x not in Teams_Remove]
    df_Games = clean_df(df_Games, Teams, Teams2, week)
    #Wins = get_Wins(team, df_Games)
    df_T_Wins = get_df_T_Wins(Teams, df_Games)
    Poll = get_Final_AP_Poll(year)
    A = []
    B = []
    C = []
    D = []
    #print(df_T_Wins['Team'])
    #print(Poll['Team'])
    list1 = df_T_Wins['Team'].tolist()
    list2 = Poll['Team'].tolist() 
    for team in list1:
        if team in list2:
            ind = list2.index(team)
            A.append(Poll['Rank'][ind]) 
            #B.append(Poll['Record'][ind]) 
            C.append(Poll['1st'][ind])
        else:
            A.append('NR') 
            #B.append('')
            C.append('')
        record = get_Record(team,df_Games)
        record_All = get_Record(team,df_Games_All)
        B.append(record)
        D.append(record_All)
    df_T_Wins['Rank'] = A
    df_T_Wins['FBS Record'] = B
    df_T_Wins['Full Record'] = D
    df_T_Wins['1st'] = C

    return(df_T_Wins)

In [ ]:
def get_Team_info(Team, year):
    week = 30
    Teams = get_Teams(year)
    df_Games = get_Games(year)
    Teams2 = get_Teams2(df_Games)
    df_Games = clean_df(df_Games, Teams, Teams2, week)
    Wins = get_Wins(Team, df_Games)
    T_Wins = get_T_wins(Team, df_Games)
    Record = get_Record(Team, df_Games)
    print("Team: ", Team, year)
    print("Record: ", Record)
    print("Wins (",len(Wins),"):", Wins)
    print("Transitive Wins (",len(T_Wins),"/",len(Teams)-1,"):", T_Wins)
    Missing = []
    for team in Teams:
        if team not in T_Wins and team != Team:
            Missing.append(team)
    print("Missing:",Missing)


In [ ]:
def get_Differential_Tier(df_Games, Team, Best_Win):
    score = 0
    for winner, loser, P1, P2 in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Pts'], df_Games['Pts.1']):
        if winner == Team:
            num = Tier_dict.get(loser)
            if num == Best_Win:
                score = score + (int(P1) - int(P2))
        if loser == Team:
            num = Tier_dict.get(winner)
            if num == Best_Win:
                score = score - (int(P1) - int(P2))
    return(score)





In [ ]:
def get_Differential(df_Games, Team):
    Points_Scored = 0
    Points_Allowed = 0
    for winner, loser, P1, P2 in zip(df_Games['Winner'], df_Games['Loser'], df_Games['Pts'], df_Games['Pts.1']):
        if P1:
            if winner == Team:
                #score = score + (int(P1) - int(P2))
                Points_Scored = Points_Scored + int(P1)
                Points_Allowed = Points_Allowed + int(P2)
            if loser == Team:
                #score = score - (int(P1) - int(P2))
                Points_Scored = Points_Scored + int(P2)
                Points_Allowed = Points_Allowed + int(P1)
    Differential = Points_Scored - Points_Allowed
    return(Differential, Points_Scored, Points_Allowed)

In [ ]:
def Rankings_Compare(List1, List2):
    Placement = list(range(1,26))
    dict1 = {List1[i] : Placement[i] for i in range(len(List1))}
    dict2 = {List2[i] : Placement[i] for i in range(len(List2))}
    d = 0
    for team in List1:
        a = dict1.get(team)
        b = dict2.get(team,26)
        nd = b - a
        d = d + abs(nd)
  #      print(team)
  #      print(nd)
    for team in List2:
        if team not in List1:
            a = dict2.get(team)
            b = dict1.get(team,26)
            nd = a - b
            d = d + abs(nd)
  #          print(team)
  #          print(nd)
    ad = d/25
    return(d, ad)

In [ ]:
def get_Team_scores(Team, FBS, df_Games):

    i = 1
    Wins2 = 0
    new_list = []
    I_new = []
    team_list = get_Wins(Team,df_Games)
    Wins_List = team_list
  
    for team in team_list:
        if team in FBS:
            new_list.append(team)
    team_list = new_list

    for team in team_list:
        num = get_Wins(team, df_Games)
        Wins2 = Wins2 + len(num)
    Wins = len(team_list)
    new_list = []
    length = len(team_list)
    I = [1] * length
    diff = team_list
    while length != 0:
        i = i + 1
        for team in diff:
            tlist = get_Wins(team,df_Games)
            for team2 in tlist:
                if team2 not in team_list: 
                    if team2 not in new_list:
                        if team2 in FBS:
                            if team2 != Team:
                                new_list.append(team2)
                                I_new.append(i)
        length = len(new_list)
        diff = new_list
        team_list = team_list + new_list 
        I = I + I_new
        new_list = []
        I_new = []
    T_Wins = len(team_list)
    score = 0
    for point in I:
        score = score + (.5 ** (point - 1))
    
    return(Wins_List, Wins, Wins2, I, T_Wins, score)

In [ ]:
def get_FCS(df_Games, Team, FBS):
    FCS_Wins = []
    FCS_Losses = []
    for winner, loser in zip(df_Games['Winner'], df_Games['Loser']):
        if winner == Team and loser not in FBS:
            FCS_Wins.append(loser)
        if loser == Team and winner not in FBS:
            FCS_Losses.append(winner)
    return(FCS_Wins, FCS_Losses)

In [ ]:
def get_FBS(df_Games, Teams, Teams2):
    Teams_List = []
    for team in Teams:
        if team in Teams2:
            Teams_List.append(team)
    #Diff = []
    #for team in Teams2:
    #    if team not in Teams:
    #        Diff.append(team)
    #for team in Teams:
    #    if team not in Teams2:
    #        Teams.remove(team)
    return(Teams_List)

In [ ]:
def get_Tiers(T_Wins):
    T_Wins_Unique = []
    Tier_List = []
    for t in T_Wins:
        if t not in T_Wins_Unique:
            T_Wins_Unique.append(t)
    T_Wins_Unique.sort(reverse = True)

    l = len(T_Wins_Unique)
    Tiers = list(range(1,l+1))

    for num in T_Wins:
        for num1, num2 in zip(T_Wins_Unique, Tiers):
            if num == num1:
                Tier_List.append(num2)
    return(Tier_List)


In [ ]:
def get_Ranking(Lists):
    List_Sorted = (Lists[:])
    List_Sorted = list(map(float, List_Sorted))
    List_Sorted.sort(reverse = True)
    Ranking = []
    i = 1
    R = 1
    item = List_Sorted[0]    
    Ranks = [R]
    items = [item]
    for L in List_Sorted:
        if L != item:
            R = i
            item = L
            Ranks.append(R)
            items.append(item)
        i = i + 1

    Rank_dict = {items[i] : Ranks[i] for i in range(len(items))}
    for L in Lists:
        A = Rank_dict[float(L)]
        Ranking.append(A)
    return(Ranking)

In [ ]:
def get_Tier_data(df_Games, df_Games_All, Team, Tier_dict):
    Tier_Wins = []
    Tier_Losses = []
    Best_Wins = []
    Worst_Losses = []
    Win_Counts = []
    Loss_Counts = []
    Tier_Values = Tier_dict.values()
    FCS = max(Tier_Values) + 1
    for team in Team:
        Tier_Win = []
        Tier_Loss = []
        FCS_Win, FCS_Loss = get_FCS(df_Games_All, team, Team)
        if FCS_Win:
            Tier_Win.append(FCS)
            Tier_Win * len(FCS_Win)
        if FCS_Loss:
            Tier_Loss.append(FCS)
            Tier_Loss * len(FCS_Loss)
        Best_Win = 99
        Worst_Loss = -1
        Win_Count = 0
        Loss_Count = 0
        win_list = get_Wins(team, df_Games)
        loss_list = get_Losses(team, df_Games)
        if win_list or FCS_Win:
            for team in win_list:
                num = Tier_dict.get(team)
                Tier_Win.append(num)
            Best_Win = min(Tier_Win)
            Win_Count = Tier_Win.count(Best_Win)
        Tier_Wins.append(Tier_Win)
        Best_Wins.append(Best_Win)
        Win_Counts.append(Win_Count)

        if loss_list or FCS_Loss:
            for team in loss_list:
                num = Tier_dict.get(team)
                Tier_Loss.append(num)
            Worst_Loss = max(Tier_Loss)
            Loss_Count = Tier_Loss.count(Worst_Loss)
        Tier_Losses.append(Tier_Loss)
        Worst_Losses.append(Worst_Loss)
        Loss_Counts.append(Loss_Count)    

    return(Best_Wins, Win_Counts, Worst_Losses, Loss_Counts)                  

## Historical Data

In [ ]:
column_names = ["Year","Team", "#", "Transitive Wins","Missing"]
#df_1937 = get_Transitive_Wins(1937)
df_Champions = pd.DataFrame(columns = column_names)
#df_Champions = df_1937.iloc[0:1,:]
#years = range(1900,2021)
years_remove = [1871,1967,1989,2003,2007,2014,2015,2016]
df_dict = {}

for year in range(1869,2021):
    if year not in years_remove:
        clear_output(wait=True)
        print(year)
        week = 30
        df_T_Wins = get_Transitive_Wins(year)
        df_T_Wins['Year'] = year
        exec('df_T_Wins_{} = df_T_Wins'.format(year))
        max = df_T_Wins.iloc[0,1]
        i = 0
        for num in df_T_Wins['#']:
            if num == max:
                i = i+1
        new_df = df_T_Wins.iloc[0:i,:]
        df_Champions = df_Champions.append(new_df, ignore_index = True)
pd.set_option("display.max_rows", None, "display.max_columns", None)
df_Champions

2020


,Year,Team,#,Transitive Wins,Missing,Rank,FBS Record,Full Record,1st
0,1869,Rutgers,1,[Princeton],[],NR,(1–1),(1–1),
1,1869,Princeton,1,[Rutgers],[],NR,(1–1),(1–1),
2,1870,Princeton,2,"[Rutgers, Columbia]",[],NR,(1–0),(1–0),
3,1872,Princeton,2,"[Rutgers, Columbia]",[Yale],NR,(1–0),(1–0),
4,1873,Princeton,3,"[Yale, Rutgers, Columbia]",[NYU],NR,(1–0),(1–0),
5,1874,Princeton,2,"[Columbia, Rutgers]","[NYU, Yale]",NR,(2–0),(2–0),
6,1874,Yale,2,"[Columbia, Rutgers]","[Princeton, NYU]",NR,(2–0),(3–0),
7,1875,Princeton,3,"[Columbia, Yale, Rutgers]","[NYU, Harvard]",NR,(1–0),(2–0),
8,1876,Yale,4,"[Harvard, Princeton, Columbia, Penn]","[Rutgers, NYU]",NR,(3–0),(3–0),
9,1877,Princeton,3,"[Harvard, Columbia, Rutgers]",[Yale],NR,(2–0–1),(2–0–1),


In [ ]:
Champs = df_Champions['Team']
Champs.value_counts()

Yale                19
Princeton           17
Notre Dame          14
Alabama              8
Harvard              8
Penn                 6
Oklahoma             6
Ohio State           5
Auburn               5
Nebraska             5
Army                 4
Michigan             4
Rutgers              4
Minnesota            4
Georgia              4
Georgia Tech         4
USC                  4
Miami (FL)           3
Pittsburgh           3
LSU                  3
Tennessee            3
California           3
Clemson              3
Arizona State        3
Tennessee State      3
Texas                3
Florida State        3
Texas A&M            2
Illinois             2
UCLA                 2
Ole Miss             2
Fresno State         2
Chicago              2
SMU                  2
Miami (OH)           2
Penn State           1
Brown                1
Buffalo              1
Dartmouth            1
New Mexico State     1
Navy                 1
BYU                  1
UCF                  1
Fordham    

In [ ]:
column_names = ["Year","Team", "#", "Transitive Wins","Missing","#2","check"]
#df_1937 = get_Transitive_Wins(1937)
df_non_Champions = pd.DataFrame(columns = column_names)
#df_Champions = df_1937.iloc[0:1,:]
#years = range(1937,2021)
years = [1967,1989,2003,2007,2014,2015,2016]

for year in years:
    clear_output(wait=True)
    print(year)
    week = 30
    df_T_Wins = get_Transitive_Wins(year)
    df_T_Wins['Year'] = year
    exec('df_T_Wins_{} = df_T_Wins'.format(year))
    max = df_T_Wins.iloc[0,1]
    i = 0
    for num in df_T_Wins['#']:
        if num != max:
            i = i+1
    new_df = df_T_Wins.tail(i)
    df_non_Champions = df_non_Champions.append(new_df, ignore_index = True)
df_non_Champions


2016


,Year,Team,#,Transitive Wins,Missing,#2,check,Rank,FBS Record,Full Record,1st
0,1967,Penn,4,"[Brown, Columbia, Colgate, Rutgers]","[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(2–5),(3–6),
1,1967,Brown,3,"[Colgate, Columbia, Rutgers]","[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(2–5–1),(2–6–1),
2,1967,Columbia,2,"[Colgate, Rutgers]","[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(2–7),(2–7),
3,1967,San Jose State,1,[New Mexico],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(1–6),(2–7),
4,1967,Pittsburgh,1,[Wisconsin],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(1–9),(1–9),
5,1967,Rutgers,1,[Colgate],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(1–4),(4–5),
6,1967,Wisconsin,0,[],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(0–9–1),(0–9–1),
7,1967,Davidson,0,[],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(0–5),(4–5),
8,1967,Idaho,0,[],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(0–3),(4–6),
9,1967,Colgate,0,[],"[West Virginia, Villanova, William & Mary, Hou...",NaN,NaN,NR,(0–7),(2–8),


In [ ]:
column_names = ["Year","Rank", "Team", "Record","1st"]
df_AP_Champions = pd.DataFrame(columns = column_names)
for year in range(1936,2021):
    df_Final_AP_Poll = get_Final_AP_Poll(year)
    new_df = df_Final_AP_Poll.iloc[0:i,:]
    df_AP_Champions = df_AP_Champions.append(new_df, ignore_index = True)
    print(df_Final_AP_Poll)
    print('--------------------------------------------------')


    Year  Rank          Team    Record    1st
0   1936     1     Minnesota     (7–1)   (25)
1   1936     2           LSU   (9–0–1)    (9)
2   1936     3    Pittsburgh   (7–1–1)       
3   1936     4       Alabama   (8–0–1)       
4   1936     5    Washington   (7–1–1)       
5   1936     6   Santa Clara     (7–0)    (1)
6   1936     7  Northwestern     (7–1)       
7   1936     8    Notre Dame     (6–2)       
8   1936     9      Nebraska     (7–2)       
9   1936    10          Penn     (7–1)       
10  1936    11          Duke     (9–1)       
11  1936    12          Yale     (7–1)       
12  1936    13     Dartmouth   (7–1–1)       
13  1936    14      Duquesne     (7–2)       
14  1936    15       Fordham   (5–1–2)       
15  1936    16           TCU   (7–2–2)       
16  1936    17     Tennessee   (6–2–1)       
17  1936    18      Arkansas    (6–3)        
18  1936    18          Navy    (6–3)        
19  1936    20     Marquette     (7–1)       
----------------------------------

In [ ]:
df_AP_Champions

,Year,Rank,Team,Record,1st
0,1936,1,Minnesota,(7–1),(25)
1,1937,1,Pittsburgh,(9–0–1),(30)
2,1938,1,TCU,(10–0),(55)
3,1939,1,Texas A&M,(10–0),(55)
4,1940,1,Minnesota,(8–0),(65)
5,1941,1,Minnesota,(8–0),(84.5)
6,1942,1,Ohio State,(9–1),(84)
7,1943,1,Notre Dame,(9–1),(86)
8,1944,1,Army,(9–0),(95)
9,1945,1,Army,(9–0),(115)


In [ ]:
AP_Champs = df_AP_Champions['Team']
AP_Champs.value_counts()

Alabama              11
Oklahoma              7
Notre Dame            7
Ohio State            5
USC                   5
Miami (FL)            4
Clemson               3
Florida               3
Nebraska              3
Texas                 3
Minnesota             3
LSU                   3
Florida State         3
Tennessee             2
Penn State            2
Michigan              2
Auburn                2
Army                  2
Pittsburgh            2
Nebraska (13–0)       1
TCU                   1
Notre Dame (12–0)     1
Texas A&M             1
Maryland              1
Syracuse              1
Alabama (10–0)        1
Georgia (12–0)        1
Minnesota (8–1)       1
Miami (FL) (11–1)     1
Colorado              1
Michigan State        1
BYU                   1
Name: Team, dtype: int64

In [ ]:
df_AP_T_Champs = df_Champions[df_Champions['Year'] >= 1936]
AP_T_Champs = df_AP_T_Champs['Team']
AP_T_Champs.value_counts()


Notre Dame          10
Alabama              6
Oklahoma             6
Auburn               5
Nebraska             5
Ohio State           5
Arizona State        3
Tennessee State      3
USC                  3
Miami (FL)           3
Clemson              3
Texas                3
Georgia              3
Florida State        3
Rutgers              3
Army                 3
LSU                  3
Michigan             2
Ole Miss             2
Georgia Tech         2
Tennessee            2
Fresno State         2
Texas A&M            2
Miami (OH)           2
Memphis              1
Air Force            1
BYU                  1
Buffalo              1
Yale                 1
Fordham              1
Dartmouth            1
Pittsburgh           1
Syracuse             1
New Mexico State     1
Penn State           1
Hardin–Simmons       1
Bowling Green        1
VMI                  1
Princeton            1
Oklahoma State       1
Arkansas State       1
William & Mary       1
UCLA                 1
Toledo     

## To Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe


gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from gspread_dataframe import set_with_dataframe
sh = gc.create('FBS Transitive Data')

worksheet = gc.open('FBS Transitive Data').sheet1
set_with_dataframe(worksheet, df_Champions)

worksheet2 = sh.add_worksheet(title="1869", rows="150", cols="10")
worksheet3 = sh.add_worksheet(title="1870", rows="150", cols="10")
for num in range(4,153):
    year = num + 1868
    exec('worksheet{} = sh.add_worksheet(title="{}", rows="150", cols="10")'.format(num,year))

In [ ]:
set_with_dataframe(worksheet2, df_T_Wins_1869)
set_with_dataframe(worksheet3, df_T_Wins_1870)
for num in range(4,153):
    year = num + 1868
    exec('set_with_dataframe(worksheet{}, df_T_Wins_{})'.format(num,year))



NameError: ignored

### Weekly Data

In [ ]:
from gspread_dataframe import set_with_dataframe
sh = gc.create('FBS Transitive Data 2021')

worksheet = gc.open('FBS Transitive Data 2021').sheet1
set_with_dataframe(worksheet, df_team_scores)

NameError: ignored

In [ ]:
from gspread_dataframe import set_with_dataframe

worksheet = gc.open('FBS Transitive Data 2021')

worksheet2 = worksheet.add_worksheet(title="Final Rankings", rows="131", cols="8")
set_with_dataframe(worksheet2, df_team_scores)

worksheet2 = worksheet.add_worksheet(title="Final Rankings Raw Data", rows="131", cols="8")
set_with_dataframe(worksheet2, df_team_scores_raw)

#worksheet2 = worksheet.add_worksheet(title="Week 3", rows="131", cols="14")
#set_with_dataframe(worksheet2, df_team_scores)
#worksheet2 = worksheet.add_worksheet(title="Week 4", rows="131", cols="12")
#set_with_dataframe(worksheet2, df_team_scores)
#worksheet2 = worksheet.add_worksheet(title="Week 5", rows="131", cols="12")
#set_with_dataframe(worksheet2, df_team_scores)
#worksheet2 = worksheet.add_worksheet(title="Week 6", rows="131", cols="14")
#set_with_dataframe(worksheet2, df_team_scores)
#worksheet2 = worksheet.add_worksheet(title="Week 7", rows="131", cols="14")
#set_with_dataframe(worksheet2, df_team_scores)

## Search

In [ ]:
get_Teams(2020)

['Air Force',
 'Akron',
 'Alabama',
 'Appalachian State',
 'Arizona',
 'Arizona State',
 'Arkansas',
 'Arkansas State',
 'Army',
 'Auburn',
 'Ball State',
 'Baylor',
 'Boise State',
 'Boston College',
 'Bowling Green',
 'Buffalo',
 'BYU',
 'California',
 'Central Michigan',
 'Charlotte',
 'Cincinnati',
 'Clemson',
 'Coastal Carolina',
 'Colorado',
 'Colorado State',
 'Duke',
 'East Carolina',
 'Eastern Michigan',
 'FIU',
 'Florida',
 'Florida Atlantic',
 'Florida State',
 'Fresno State',
 'Georgia',
 'Georgia Southern',
 'Georgia State',
 'Georgia Tech',
 'Hawaii',
 'Houston',
 'Illinois',
 'Indiana',
 'Iowa',
 'Iowa State',
 'Kansas',
 'Kansas State',
 'Kent State',
 'Kentucky',
 'Liberty',
 'Louisiana',
 'Louisiana–Monroe',
 'Louisiana Tech',
 'Louisville',
 'LSU',
 'Marshall',
 'Maryland',
 'Massachusetts',
 'Memphis',
 'Miami (FL)',
 'Miami (OH)',
 'Michigan',
 'Michigan State',
 'Middle Tennessee',
 'Minnesota',
 'Mississippi State',
 'Missouri',
 'Navy',
 'Nebraska',
 'Nevada',
 

In [ ]:
get_Team_info("Alabama", 2020)

Team:  Alabama 2020
Record:  (13–0)
Wins ( 13 ): ['Missouri', 'Texas A&M', 'Ole Miss', 'Georgia', 'Tennessee', 'Mississippi State', 'Kentucky', 'Auburn', 'LSU', 'Arkansas', 'Florida', 'Notre Dame', 'Ohio State']
Transitive Wins ( 113 / 127 ): ['Missouri', 'Texas A&M', 'Ole Miss', 'Georgia', 'Tennessee', 'Mississippi State', 'Kentucky', 'Auburn', 'LSU', 'Arkansas', 'Florida', 'Notre Dame', 'Ohio State', 'South Carolina', 'Vanderbilt', 'North Carolina', 'Indiana', 'Cincinnati', 'Tulsa', 'North Carolina State', 'Duke', 'South Florida', 'Florida State', 'Louisville', 'Pittsburgh', 'Georgia Tech', 'Clemson', 'Boston College', 'Syracuse', 'Nebraska', 'Penn State', 'Rutgers', 'Michigan State', 'Northwestern', 'Virginia Tech', 'Wake Forest', 'Miami (FL)', 'Michigan', 'Maryland', 'Wisconsin', 'Army', 'SMU', 'Memphis', 'Houston', 'East Carolina', 'UCF', 'Tulane', 'Navy', 'Virginia', 'Liberty', 'Charlotte', 'Western Kentucky', 'Texas State', 'Purdue', 'Illinois', 'Iowa', 'UAB', 'Minnesota', 'Midd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
df = get_Transitive_Wins(2020)
df

IndexError: ignored

In [ ]:
#for year in range(1941,2021):
df_Final_AP_Poll = get_Final_AP_Poll(1937)
#print(year)
df_Final_AP_Poll

df_Final_AP_Poll['Record'][5]

' (8–0–1)'

In [ ]:
get_Games(2020)

,Wk,Date,Day,Winner,Pts,@,Loser,Pts.1
0,1,"Sep 3, 2020",Thu,UAB,45,None,Central Arkansas,35
1,1,"Sep 3, 2020",Thu,South Alabama,32,@,Southern Miss,21
2,1,"Sep 5, 2020",Sat,Army,42,None,Middle Tennessee,0
3,1,"Sep 5, 2020",Sat,Marshall,59,None,Eastern Kentucky,0
4,1,"Sep 5, 2020",Sat,Memphis,37,None,Arkansas State,24
5,1,"Sep 5, 2020",Sat,North Texas,57,None,Houston Baptist,31
6,1,"Sep 5, 2020",Sat,SMU,31,@,Texas State,24
7,1,"Sep 5, 2020",Sat,UTEP,24,None,Stephen F. Austin,14
8,1,"Sep 7, 2020",Mon,BYU,55,@,Navy,3
9,2,"Sep 10, 2020",Thu,Miami (FL),31,None,UAB,14


In [ ]:
get_Team_info("Notre Dame", 2018)

Team:  Notre Dame 2018
Record:  (12–1)
Wins ( 12 ): ['Michigan', 'Ball State', 'Vanderbilt', 'Wake Forest', 'Stanford', 'Virginia Tech', 'Pittsburgh', 'Navy', 'Northwestern', 'Florida State', 'Syracuse', 'USC']
Transitive Wins ( 127 / 129 ): ['Michigan', 'Ball State', 'Vanderbilt', 'Wake Forest', 'Stanford', 'Virginia Tech', 'Pittsburgh', 'Navy', 'Northwestern', 'Florida State', 'Syracuse', 'USC', 'Western Michigan', 'SMU', 'Nebraska', 'Maryland', 'Wisconsin', 'Michigan State', 'Penn State', 'Rutgers', 'Indiana', 'Kent State', 'Central Michigan', 'Middle Tennessee', 'Nevada', 'Arkansas', 'Ole Miss', 'Tennessee', 'Tulane', 'Rice', 'Louisville', 'North Carolina State', 'Duke', 'Memphis', 'San Diego State', 'Oregon', 'Arizona State', 'Oregon State', 'UCLA', 'California', 'North Carolina', 'Virginia', 'Marshall', 'Georgia Tech', 'Tulsa', 'Purdue', 'Iowa', 'Minnesota', 'Illinois', 'Utah', 'Northern Illinois', 'Boston College', 'UConn', 'West Virginia', 'UNLV', 'Washington State', 'Arizona',

In [ ]:
df_Games_2020 = get_Games(2020)
FBS_2020 = get_Teams(2020)
FBS = FBS_2020

In [ ]:
get_Team_info("Florida Atlantic", 2020)

Team:  Florida Atlantic 2020
Record:  (5–4)
Wins ( 5 ): ['Charlotte', 'UTSA', 'Western Kentucky', 'FIU', 'Massachusetts']
Transitive Wins ( 108 / 127 ): ['Charlotte', 'UTSA', 'Western Kentucky', 'FIU', 'Massachusetts', 'North Texas', 'UTEP', 'Texas State', 'Middle Tennessee', 'Louisiana Tech', 'Southern Miss', 'Rice', 'Louisiana–Monroe', 'Arkansas State', 'Troy', 'UAB', 'Marshall', 'Kansas State', 'Georgia State', 'South Alabama', 'Appalachian State', 'Oklahoma', 'Texas Tech', 'TCU', 'Kansas', 'East Carolina', 'Georgia Southern', 'Texas', 'Oklahoma State', 'Baylor', 'Iowa State', 'Florida', 'West Virginia', 'South Florida', 'Temple', 'SMU', 'Colorado', 'Tulsa', 'Miami (FL)', 'Oregon', 'Ole Miss', 'South Carolina', 'Missouri', 'Georgia', 'Arkansas', 'Vanderbilt', 'Kentucky', 'Tennessee', 'Army', 'Memphis', 'Tulane', 'Navy', 'UCLA', 'Stanford', 'San Diego State', 'Arizona', 'UCF', 'Louisville', 'Florida State', 'Pittsburgh', 'Virginia', 'North Carolina State', 'Virginia Tech', 'Duke', 'W

##Transitive Dataframe

In [ ]:
year = 2021
week = 20

Team = []
Wins_Count = []
Losses_Count = []
Ties_Count = []
T_Wins_Count = []
T_Losses_Count = []
Best_Wins_Tier = []
Worst_Losses_Tier = []
WL_Tiers = []
Best_Wins_Count = []
Worst_Losses_Count = []
Win_Diff = []
Loss_Diff = []
Total_Diff = []
Points_Scored = []
Points_Allowed = []
Cluster_Differential = []
T_Wins_List = []
T_Losses_List = []
Clusters = []
Cluster_Win_Percentage = []
Cluster_Adjusted_Win_Percentage = []
Total_Adjusted_Win_Percentage = []

Transitive_Score = []
Transitive_Score_2 = []
SOR = []
SOS = []
ROS = []
AROS = []
ROSP = []
AROSP = []
ROS_Remaining = []
ROSRP = []
Team_Record = []

df_Games_All = get_Games(year)
Teams = get_Teams(year)
Teams2 = get_Teams2(df_Games_All)
if year > 1936:
    FBS = get_FBS(df_Games_All, Teams, Teams2)
else:
    FBS = Teams2
if year == 2020:
    remove = ["UConn", "Old Dominion", "New Mexico State"]
    FBS = [ele for ele in FBS if ele not in remove]
Teams2 = get_Teams2(df_Games_All)
df_Games_All['Wk'] = df_Games_All['Wk'].astype(int) 
df_Games_All_Played = df_Games_All[df_Games_All['Wk'] <= week]
if year > 1936:
    df_Games = clean_df(df_Games_All, Teams, Teams2, week)
else:
    df_Games = df_Games_All

for team in FBS:
    Wins = get_Wins(team, df_Games)
    Losses = get_Losses(team, df_Games)
    Ties = get_Ties(team, df_Games)
    T_Wins = get_T_wins(team,df_Games)
    T_Losses = get_T_losses(team,df_Games)
    Cluster = get_Cluster(team, df_Games)
    Team.append(team)
    Wins_Count.append(len(Wins))
    Losses_Count.append(len(Losses))
    Ties_Count.append(len(Ties))
    T_Wins_Count.append(len(T_Wins))
    T_Losses_Count.append(len(T_Losses))
    Transitive_Score.append((len(T_Wins))-((len(T_Losses))))
    T_Wins_List.append(T_Wins)
    Clusters.append(Cluster)
    T_Losses_List.append(T_Losses)
Win_dict = {Team[i] : Wins_Count[i] for i in range(len(Team))}
Loss_dict = {Team[i] : Losses_Count[i] for i in range(len(Team))}
trans_win_dict = {Team[i] : T_Wins_Count[i] for i in range(len(Team))}
trans_loss_dict = {Team[i] : T_Losses_Count[i] for i in range(len(Team))}

Tiers = get_Tiers(Transitive_Score)
Tier_dict = {Team[i] : Tiers[i] for i in range(len(Team))}

T_Win_dict = {Team[i] : T_Wins_Count[i] for i in range(len(Team))}
T_Loss_dict = {Team[i] : T_Losses_Count[i] for i in range(len(Team))}

Best_Wins_Tier, Best_Wins, Worst_Losses_Tier, Worst_Losses =  get_Tier_data(df_Games, df_Games_All_Played, FBS, Tier_dict)
for team, C, BW, WL, BWT, WLT in zip(Team, Clusters, Best_Wins, Worst_Losses, Best_Wins_Tier, Worst_Losses_Tier):
    #Differential1 = get_Differential_Tier(df_Games, team, BWT)
    #Differential2 = get_Differential_Tier(df_Games, team, WLT)
    (Differential, PS, PA) = get_Differential(df_Games, team)
    #Win_Diff.append(Differential1)
    #Loss_Diff.append(Differential2)
    Total_Diff.append(Differential)
    Points_Scored.append(PS)
    Points_Allowed.append(PA)
    WL_Tiers.append(str(BWT)+"/"+str(WLT))
    if C == True:
        CWP = BW/(BW+WL)
        CAWP = (BW +1 )/(BW + WL +2)
        CDiff = Differential
    if C == False:
        CWP = 999
        CAWP = 999
        CDiff = 999

    Cluster_Adjusted_Win_Percentage.append(CAWP)
    Cluster_Differential.append(CDiff)

    Wins_list = get_Wins(team, df_Games)
    Losses_list = get_Losses(team, df_Games)
    Ties_list = get_Ties(team, df_Games)
    Wins = len(Wins_list)
    Losses = len(Losses_list)
    Ties = len(Ties_list)
    WP = 0
    if (Wins+Losses) > 0:
       WP = (Wins)/(Wins + Losses)
    AWP = (Wins + 1)/(Wins + Losses + 2)
    Total_Adjusted_Win_Percentage.append(AWP)
    if Ties:
        R = str(Wins)+'-'+str(Losses)+'-'+str(Ties)
    else:    
        R = str(Wins)+'-'+str(Losses)
    Team_Record.append(R)

    T2 = 0
    T3 = 0
    W = 0
    L = 0
    W2 = 0
    L2 = 0
    W3 = 0
    L3 = 0
    for t in Wins_list:
        T2 = T2 + T_Win_dict[t]
        T3 = T3 + Win_dict[t]
        W = W + Win_dict[t]
        L = L + Loss_dict[t]
        W2 = W2 + Win_dict[t]
        L2 = L2 + Loss_dict[t] -1


    for t in Losses_list:
        T2 = T2 - T_Loss_dict[t]
        T3 = T3 - Loss_dict[t]
        W = W + Win_dict[t]
        L = L + Loss_dict[t]
        W2 = W2 + Win_dict[t] -1
        L2 = L2 + Loss_dict[t]

    Transitive_Score_2.append(T2)
    SOR.append(T3)
    SOS.append(W)
    R = str(W)+'-'+str(L)
    RP = '0'
    A = '0'
    AP = '0'
    if W+L > 0:
        RP = str(round((W/(W+L)),3))
        A = str(W2)+'-'+str(L2)
        AP = str(round((W2/(W2+L2)),3))
    ROS.append(R)
    ROSP.append(RP)
    AROS.append(A)
    AROSP.append(AP)

    R2 = None
    Remain = get_Remaining(team, df_Games_All, week, FBS)
    for t in Remain:
        W3 = W3 + Win_dict[t]
        L3 = L3 + Loss_dict[t]
    if Remain:
        R2 = str(W3)+'-'+str(L3)
        RP2 = str(round((W3/(W3+L3)),3))
    else: 
        R2 = 0
        RP2 = 0
    ROS_Remaining.append(R2)
    ROSRP.append(RP2)
Team_num = len(FBS)
Rank = list(range(1,Team_num + 1))
SOR_Rank = get_Ranking(SOR)
Total_Diff_Rank = get_Ranking(Total_Diff)
ROSP_Rank = get_Ranking(ROSP)
AROSP_Rank = get_Ranking(AROSP)
ROSRP_Rank = get_Ranking(ROSRP)
NROSPR = []
NROSR = []

for RRP, RR in zip(ROSRP, ROS_Remaining):
    if RR == 0:
        NROSPR.append(None)
        NROSR.append(None)
    else:
        NROSPR.append(RRP)
        NROSR.append(RR)
ROSRP = NROSPR
ROS_Remaining = NROSR

df_team_scores_raw = pd.DataFrame(Rank,columns = ['Rank'])
df_team_scores_raw['Team'] = Team
df_team_scores_raw['Record'] = Team_Record

#df_team_scores['Team (FBS Record)'] = Team_Record
df_team_scores_raw['Tier'] = Tiers
df_team_scores_raw['Transitivity Score'] = Transitive_Score
#df_team_scores['Transitive Wins'] = T_Wins_Count
#df_team_scores['Transitive Losses'] = T_Losses_Count
df_team_scores_raw['Best Win Tier'] = Best_Wins_Tier
df_team_scores_raw['Worst Loss Tier'] = Worst_Losses_Tier
#df_team_scores_raw['Best Win/Worst Loss Tier'] = WL_Tiers
#df_team_scores['Tier Wins'] = Best_Wins
#df_team_scores['Tier Losses'] = Worst_Losses
#df_team_scores['Best Win Tier Point Differential'] = Win_Diff
#df_team_scores['Worst Loss Tier Point Differential'] = Loss_Diff
#df_team_scores['Cluster Adjusted Win Percentage'] = Cluster_Adjusted_Win_Percentage
#df_team_scores['Record'] = Record
#df_team_scores['Record of Schedule'] = ROS
df_team_scores_raw['Strength of Record'] = SOR
df_team_scores_raw['Strength of Record Rank'] = SOR_Rank

df_team_scores_raw['Points Scored'] = Points_Scored
df_team_scores_raw['Points Allowed'] = Points_Allowed
df_team_scores_raw['Total Point Differential'] = Total_Diff
df_team_scores_raw['Total Point Differential Rank'] = Total_Diff_Rank

df_team_scores_raw['Record of Schedule'] = ROS
df_team_scores_raw['Record of Schedule %'] = ROSP
df_team_scores_raw['Record of Schedule Rank'] = ROSP_Rank

df_team_scores_raw['Adjusted Record of Schedule'] = AROS
df_team_scores_raw['Adjusted Record of Schedule %'] = AROSP
df_team_scores_raw['Adjusted Record of Schedule Rank'] = AROSP_Rank

df_team_scores_raw['Record of Schedule Remaining'] = ROS_Remaining
df_team_scores_raw['Record of Schedule Remaining %'] = ROSRP
df_team_scores_raw['Record of Schedule Remaining Rank'] = ROSRP_Rank

#df_team_scores['Total Adjusted Win Percentage'] = Total_Adjusted_Win_Percentage
#df_team_scores['Transitivity Record Score'] = SOR
#df_team_scores['Strength of Schedule'] = SOS

#df_team_scores['Cluster Point Differential'] = Cluster_Differential
df_team_scores_raw['Transitive Wins'] = T_Wins_Count
df_team_scores_raw['Transitive Losses'] = T_Losses_Count

df_team_scores_raw['Transitive Wins List'] = T_Wins_List
df_team_scores_raw['Transitive Losses List'] = T_Losses_List

df_team_scores_raw['Cluster'] = Clusters

#df_team_scores['Wins List'] = team_list
#df_team_scores['T Wins List'] = T_Wins_List
#df_team_scores['Best Win'] = Best_Wins
#df_team_scores['Number of Wins'] = Count_List
#df_team_scores['Differential'] = Diff

pd.set_option("display.max_rows", None, "display.max_columns", None)
#df_team_scores.sort_values(by= ['Transitivity Score', 'Best Win Tier', 'Worst Loss Tier', 'Cluster Adjusted Win Percentage', 'Cluster Point Differential', 'Total Adjusted Win Percentage', 'Total Point Differential'], ascending = (False, True, True, False, False, False, False), inplace = True)
#df_team_scores.sort_values(by= ['Transitivity Score', 'Transitivity Record Score', 'Best Win Tier', 'Worst Loss Tier', 'Cluster Adjusted Win Percentage', 'Cluster Point Differential', 'Total Adjusted Win Percentage', 'Total Point Differential'], ascending = (False, False, True, True, False, False, False, False), inplace = True)
#df_team_scores.sort_values(by= ['Transitivity Score', 'Best Win Tier', 'Worst Loss Tier', 'Cluster Adjusted Win Percentage', 'Total Adjusted Win Percentage', 'Transitivity Record Score', 'Cluster Point Differential', 'Total Point Differential'], ascending = (False, True, True, False, False, False, False, False), inplace = True)
#df_team_scores.sort_values(by= ['Transitivity Score', 'Best Win Tier', 'Worst Loss Tier', 'Total Adjusted Win Percentage', 'Transitivity Record Score', 'Total Point Differential'], ascending = (False, True, True, False, False, False), inplace = True)
df_team_scores_raw.sort_values(by= ['Transitivity Score', 'Best Win Tier', 'Worst Loss Tier','Strength of Record', 'Total Point Differential'], ascending = (False, True, True, False, False), inplace = True)

df_team_scores_raw.reset_index(drop = True, inplace = True)
df_team_scores_raw['Worst Loss Tier'].replace(99,'',inplace= True, regex = True)
df_team_scores_raw['Best Win Tier'].replace(-1,'',inplace= True, regex = True)
df_team_scores_raw['Worst Loss Tier'].replace(-1,'',inplace= True, regex = True)
df_team_scores_raw['Record of Schedule Remaining'].replace(0,None,inplace= True, regex = True)
df_team_scores_raw['Record of Schedule Remaining %'].replace(0,None,inplace= True, regex = True)

df_team_scores_raw['Rank'] = Rank


df_team_scores_raw.head()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: FutureWarning: The default value of regex will change from True to False in a future version.


,Rank,Team,Record,Tier,Transitivity Score,Best Win Tier,Worst Loss Tier,Strength of Record,Strength of Record Rank,Points Scored,Points Allowed,Total Point Differential,Total Point Differential Rank,Record of Schedule,Record of Schedule %,Record of Schedule Rank,Adjusted Record of Schedule,Adjusted Record of Schedule %,Adjusted Record of Schedule Rank,Record of Schedule Remaining,Record of Schedule Remaining %,Record of Schedule Remaining Rank,Transitive Wins,Transitive Losses,Transitive Wins List,Transitive Losses List,Cluster
0,1,Georgia,13-1,1,4,1,1,87,1,523,146,377,1,101-71,0.587,13,100-58,0.633,5,None,None,1,129,125,"[Clemson, UAB, South Carolina, Vanderbilt, Ark...","[Alabama, Texas A&M, Arkansas, Mississippi Sta...",True
1,2,Alabama,12-2,1,4,1,1,73,2,550,288,262,3,98-71,0.58,16,96-59,0.619,8,None,None,1,129,125,"[Miami (FL), Florida, Southern Miss, Ole Miss,...","[Texas A&M, Georgia, Arkansas, Mississippi Sta...",True
2,3,Michigan,12-2,1,4,1,1,67,3,501,243,258,4,93-78,0.544,38,91-66,0.58,18,None,None,1,129,125,"[Western Michigan, Washington, Northern Illino...","[Michigan State, Georgia, Purdue, Ohio State, ...",True
3,4,Oklahoma State,11-2,1,4,1,1,66,4,412,237,175,9,91-66,0.58,16,89-55,0.618,9,None,None,1,129,125,"[Tulsa, Boise State, Kansas State, Baylor, Tex...","[Iowa State, Baylor, Iowa, West Virginia, Texa...",True
4,5,Baylor,11-2,1,4,1,1,64,5,377,249,128,18,88-67,0.568,21,86-56,0.606,10,None,None,1,129,125,"[Texas State, Kansas, Iowa State, West Virgini...","[Oklahoma State, TCU, Iowa State, SMU, Texas, ...",True


In [ ]:
Team_Record = []
TS_Tier = []
WL_Tiers = []
SOR_Rank = []
TPD_Rank = []
AROS_Rank = []
ROSR_Rank = []
for Team, Record, Tier, TS, BWT, WLT, SOR, SORR, TPD, TPDR, AROS, AROSP, AROSR, ROSR, ROSRP, ROSRR in zip(df_team_scores_raw['Team'],df_team_scores_raw['Record'],
                          df_team_scores_raw['Tier'],df_team_scores_raw['Transitivity Score'],
df_team_scores_raw['Best Win Tier'],df_team_scores_raw['Worst Loss Tier'],
df_team_scores_raw['Strength of Record'],df_team_scores_raw['Strength of Record Rank']
,df_team_scores_raw['Total Point Differential'],df_team_scores_raw['Total Point Differential Rank']
,df_team_scores_raw['Adjusted Record of Schedule'],df_team_scores_raw['Adjusted Record of Schedule %']
,df_team_scores_raw['Adjusted Record of Schedule Rank'],df_team_scores_raw['Record of Schedule Remaining']
,df_team_scores_raw['Record of Schedule Remaining %'],df_team_scores_raw['Record of Schedule Remaining Rank']):
    Team_Record.append(Team+' '+'('+Record+')')
    TS_Tier.append(str(TS)+' '+'('+str(Tier)+')')
    WL_Tiers.append(str(BWT)+'/'+str(WLT))
    SOR_Rank.append(str(SOR)+' ('+str(SORR)+')')
    TPD_Rank.append(str(TPD)+' ('+str(TPDR)+')')
    AROS_Rank.append('('+str(AROS)+') '+str(AROSP)+' ('+str(AROSR)+')')
    if ROSRP == None:
        ROSR_Rank.append(None)
    else:
        ROSR_Rank.append('('+str(ROSR)+') '+str(ROSRP)+' ('+str(ROSRR)+')')
df_team_scores = pd.DataFrame(Rank,columns = ['Rank'])
df_team_scores['Team (FBS Record)'] = Team_Record
df_team_scores['Transitivity Score (Tier)'] = TS_Tier
df_team_scores['Best Win/Worst Loss Tier'] = WL_Tiers
df_team_scores['Strength of Record (Rank)'] = SOR_Rank
df_team_scores['Point Differential (Rank)'] = TPD_Rank
df_team_scores['Adjusted Record of Schedule (Rank)'] = AROS_Rank
df_team_scores['Record of Schedule Remaining (Rank)'] = ROSR_Rank


df_team_scores['Best Win/Worst Loss Tier'].replace('99','',inplace= True, regex = True)
df_team_scores['Best Win/Worst Loss Tier'].replace('-1','',inplace= True, regex = True)

df_team_scores['Rank'] = Rank

df_team_scores

,Rank,Team (FBS Record),Transitivity Score (Tier),Best Win/Worst Loss Tier,Strength of Record (Rank),Point Differential (Rank),Adjusted Record of Schedule (Rank),Record of Schedule Remaining (Rank)
0,1,Georgia (13-1),4 (1),1/1,87 (1),377 (1),(100-58) 0.633 (5),None
1,2,Alabama (12-2),4 (1),1/1,73 (2),262 (3),(96-59) 0.619 (8),None
2,3,Michigan (12-2),4 (1),1/1,67 (3),258 (4),(91-66) 0.58 (18),None
3,4,Oklahoma State (11-2),4 (1),1/1,66 (4),175 (9),(89-55) 0.618 (9),None
4,5,Baylor (11-2),4 (1),1/1,64 (5),128 (18),(86-56) 0.606 (10),None
5,6,Cincinnati (12-1),4 (1),1/1,63 (6),245 (5),(76-65) 0.539 (42),None
6,7,Ohio State (11-2),4 (1),1/1,58 (7),298 (2),(83-63) 0.568 (22),None
7,8,Notre Dame (11-2),4 (1),1/1,54 (8),202 (7),(78-66) 0.542 (38),None
8,9,Louisiana (12-1),4 (1),1/1,53 (9),173 (10),(64-75) 0.46 (105),None
9,10,San Diego State (11-2),4 (1),1/1,52 (10),79 (38),(75-66) 0.532 (48),None


In [ ]:
head = df_team_scores_raw.head(25)
score_list = head['Team'].tolist()
AP = get_Final_AP_Poll(year)
AP_list = AP['Team'].tolist()
print(score_list)
print(AP_list)

['Georgia', 'Alabama', 'Michigan', 'Oklahoma State', 'Baylor', 'Cincinnati', 'Ohio State', 'Notre Dame', 'Louisiana', 'San Diego State', 'Michigan State', 'UTSA', 'Oklahoma', 'Wisconsin', 'Wake Forest', 'Pittsburgh', 'Utah', 'Clemson', 'Ole Miss', 'Utah State', 'Houston', 'BYU', 'Iowa', 'Oregon', 'Appalachian State']
['Georgia', 'Alabama', 'Michigan', 'Cincinnati', 'Baylor', 'Ohio State', 'Oklahoma State', 'Notre Dame', 'Michigan State', 'Oklahoma', 'Ole Miss', 'Utah', 'Pittsburgh', 'Clemson', 'Wake Forest', 'Louisiana', 'Houston', 'Kentucky', 'BYU', 'NC State', 'Arkansas', 'Oregon', 'Iowa', 'Utah State', 'San Diego State']


In [ ]:
Rankings_Compare(score_list, AP_list)

(112, 4.48)

In [ ]:
get_Games(1900)

,Wk,Date,Day,Winner,Pts,@,Loser,Pts.1
0,1,"Sep 14, 1900",Fri,Chicago,18,None,Englewood High,0
1,1,"Sep 15, 1900",Sat,Chicago,5,None,Hyde Park High,0
2,2,"Sep 22, 1900",Sat,Beloit,23,N,Wisconsin-Whitewater,0
3,2,"Sep 22, 1900",Sat,Carlisle,34,None,Lebanon Valley,0
4,2,"Sep 22, 1900",Sat,Chicago,24,None,Lombard,0
5,2,"Sep 22, 1900",Sat,Cornell,16,None,Colgate,0
6,2,"Sep 22, 1900",Sat,Holy Cross,6,None,Massachusetts,0
7,2,"Sep 22, 1900",Sat,Knox,34,None,Carthage,0
8,2,"Sep 22, 1900",Sat,Minneapolis Central High,0,@,Minnesota,0
9,2,"Sep 22, 1900",Sat,Northwestern,26,None,North Central,0


In [ ]:
def get_Transitivity_Tree(Team, df_Games):
    Tree = []
    I = []
    I_new = []
    List1 = []
    List2 = []
    List3 = []
    Wins = get_Wins(Team, df_Games)
    for team in Wins:
        i = 0
        print(team)
        Tree.append(team)
        Team1 = team
#        Wins2 = get_Wins(team)
#        if Wins2:
#            for team in 


    team_list = get_Wins(team, df_Games)
    for team in team_list:
        num = get_Wins(team, df_Games)
    Wins = len(team_list)
    new_list = []
    length = len(team_list)
    I = [1] * length
    diff = team_list
    while length != 0:
        i = i + 1
        for team in diff:
            tlist = get_Wins(team,df_Games)
            for team2 in tlist:
                if team2 not in team_list: 
                    if team2 not in new_list:
                        if team2 in FBS:
                            if team2 != Team:
                                new_list.append(team2)
                                I_new.append(i)
                                List1.append(team2)
                                List2.append(team)
                                List3.append('('+str(i)+')'+''+team2+'-'+team)
        length = len(new_list)
        diff = new_list
        team_list = team_list + new_list 
        I = I + I_new
        new_list = []
        I_new = []

    return(List3,I)

In [ ]:
get_Transitivity_Tree('Georgia', df_Games)

Clemson
UAB
South Carolina
Vanderbilt
Arkansas
Auburn
Kentucky


(['(1)Troy-Louisiana–Monroe',
  '(1)Liberty-Louisiana–Monroe',
  '(1)Central Michigan-Missouri',
  '(1)North Texas-Missouri',
  '(1)East Carolina-South Carolina',
  '(1)Vanderbilt-South Carolina',
  '(1)Florida Atlantic-Florida',
  '(1)South Florida-Florida',
  '(1)Tennessee-Florida',
  '(1)Mississippi State-LSU',
  '(2)Southern Miss-Troy',
  '(2)Georgia Southern-Troy',
  '(2)Texas State-Troy',
  '(2)Old Dominion-Liberty',
  '(2)UAB-Liberty',
  '(2)Middle Tennessee-Liberty',
  '(2)FIU-Central Michigan',
  '(2)Ohio-Central Michigan',
  '(2)Toledo-Central Michigan',
  '(2)Marshall-East Carolina',
  '(2)Tulane-East Carolina',
  '(2)Colorado State-Vanderbilt',
  '(2)UConn-Vanderbilt',
  '(2)Bowling Green-Tennessee',
  '(2)Louisiana Tech-Mississippi State',
  '(2)North Carolina State-Mississippi State',
  '(2)Texas A&M-Mississippi State',
  '(3)Arkansas State-Georgia Southern',
  '(3)South Alabama-Texas State',
  '(3)Akron-Ohio',
  '(3)Ball State-Toledo',
  '(3)Massachusetts-Toledo',
  '(3)

In [ ]:
amax = max(Tier_dict, key=Tier_dict.get)
Tier_Values = Tier_dict.values()
FCS = max(Tier_Values) + 1
FCS
#df_team_scores.dtypes

14

In [ ]:
df1 = df_team_scores.sort_values(by= ['T Wins','T Wins 2', '2nd Wins'], ascending = (False, False, False))
df2 = df_team_scores.sort_values(by= ['T Wins','Score', 'Wins'], ascending = (False, False, False))
df3 = df_team_scores.sort_values(by= ['T Wins','2nd Wins', 'T Wins 2'], ascending = (False, False, False))

head1 = df1.head(25)
score_list1 = head1['Team'].tolist()
head2 = df2.head(25)
score_list2 = head2['Team'].tolist()
head3 = df3.head(25)
score_list3 = head3['Team'].tolist()

R1 = Rankings_Compare(score_list1, AP_list)
R2 = Rankings_Compare(score_list2, AP_list)
R3 = Rankings_Compare(score_list3, AP_list)

print(R1)
print(R2)
print(R3)

(136, 5.44)
(128, 5.12)
(136, 5.44)


In [ ]:
Team = "Ohio"
year = 2021
week = 7
Teams = get_Teams(year)
print(Teams)
df_Games = get_Games(year)
Teams2 = get_Teams2(df_Games)
df_Games = clean_df(df_Games, Teams, Teams2, week)
Wins = get_Wins(Team, df_Games)
T_Wins = get_T_wins(Team, df_Games)
Record = get_Record(Team, df_Games)
print("Team: ", Team, year)
print("Record: ", Record)
print("Wins (",len(Wins),"):", Wins)
print("Transitive Wins (",len(T_Wins),"/",len(Teams)-1,"):", T_Wins)
Missing = []
for team in Teams:
    if team not in T_Wins and team != Team:
        Missing.append(team)
print("Missing:",Missing)

['Air Force', 'Akron', 'Alabama', 'Appalachian State', 'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State', 'Army', 'Auburn', 'Ball State', 'Baylor', 'Boise State', 'Boston College', 'Bowling Green', 'Buffalo', 'BYU', 'California', 'Central Michigan', 'Charlotte', 'Cincinnati', 'Clemson', 'Coastal Carolina', 'Colorado', 'Colorado State', 'Duke', 'East Carolina', 'Eastern Michigan', 'FIU', 'Florida', 'Florida Atlantic', 'Florida State', 'Fresno State', 'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech', 'Hawaii', 'Houston', 'Illinois', 'Indiana', 'Iowa', 'Iowa State', 'Kansas', 'Kansas State', 'Kent State', 'Kentucky', 'Liberty', 'Louisiana', 'Louisiana–Monroe', 'Louisiana Tech', 'Louisville', 'LSU', 'Marshall', 'Maryland', 'Massachusetts', 'Memphis', 'Miami (FL)', 'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee', 'Minnesota', 'Mississippi State', 'Missouri', 'Navy', 'Nebraska', 'Nevada', 'New Mexico', 'New Mexico State', 'North Carolina', 'North Carolina 

In [ ]:
get_Differential_Tier(df_Games, "Air Force", 4)

NameError: ignored

In [ ]:
year = 2021
week = 5

Team = []
team_list = []
Wins = []
Wins2 = []
I = []
T_Wins = []
T_Wins2 = []
T_Wins_List = []
score = []
Diff = []

df_Games = get_Games(year)
FBS = get_Teams(year)
Teams2 = get_Teams2(df_Games)
df_Games = clean_df(df_Games, FBS, Teams2, week)

for team in FBS:
    A, B, C, D, E, F = get_Team_scores(team, FBS, df_Games)
    Team.append(team)
    team_list.append(A)
    Wins.append(B)
    Wins2.append(C)
    I.append(D)
    T_Wins.append(E)
    score.append(F)
    G = get_T_wins(team,df_Games)
    T_Wins_List.append(G)
trans_win_dict = {Team[i] : T_Wins[i] for i in range(len(Team))}
for teams in team_list:
    c = 0
    if teams:
        for team in teams:
            if trans_win_dict.get(team):
                c = c + trans_win_dict.get(team)
    T_Wins2.append(c)

# New Parameters
T_Wins_Unique = []
Tier_List = []
Tier_Wins = []
Best_Wins = []
Count_List = []
for t in T_Wins:
    if t not in T_Wins_Unique:
        T_Wins_Unique.append(t)
T_Wins_Unique.sort(reverse = True)

l = len(T_Wins_Unique)
Tiers = list(range(1,l+1))
Tiers

for num in T_Wins:
   for num1, num2 in zip(T_Wins_Unique, Tiers):
      if num == num1:
          Tier_List.append(num2)
Tier_List
T_Wins

Tier_dict = {Team[i] : Tier_List[i] for i in range(len(Team))}

for tlist in team_list:
    Tier_Win = []
    Best_Win = 99
    Count = 0
    if tlist:
        for team in tlist:
            num = Tier_dict.get(team)
            Tier_Win.append(num)
        Best_Win = min(Tier_Win)
        Count = Tier_Win.count(Best_Win)
    Tier_Wins.append(Tier_Win)
    Best_Wins.append(Best_Win)
    Count_List.append(Count)

for team, BW in zip(Team, Best_Wins):
    Differential = get_Differential_Tier(df_Games, team, BW)
    Diff.append(Differential)

        
df_team_scores = pd.DataFrame(Team,columns = ['Team'])
df_team_scores['Tier'] = Tier_List
df_team_scores['Wins List'] = team_list
df_team_scores['T Wins List'] = T_Wins_List
df_team_scores['Best Win'] = Best_Wins
df_team_scores['Number of Wins'] = Count_List
df_team_scores['Differential'] = Diff


df_team_scores['Iterations'] = I
df_team_scores['Wins'] = Wins
df_team_scores['2nd Wins'] = Wins2
df_team_scores['T Wins'] = T_Wins
df_team_scores['T Wins 2'] = T_Wins2
df_team_scores['Score'] = score

pd.set_option("display.max_rows", None, "display.max_columns", None)
df_team_scores.sort_values(by= ['T Wins','Best Win', 'Differential'], ascending = (False, True, False), inplace = True)
df_team_scores.reset_index(drop = True, inplace = True)
df_team_scores

,Team,Tier,Wins List,T Wins List,Best Win,Number of Wins,Differential,Iterations,Wins,2nd Wins,T Wins,T Wins 2,Score
0,Oregon,1,"[Fresno State, Ohio State, Arizona]","[Fresno State, Ohio State, Arizona, UConn, UCL...",2,1,7,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, ...",3,6,45,42,8.680664
1,Fresno State,2,"[UConn, UCLA, UNLV]","[UConn, UCLA, UNLV, Hawaii, LSU, Stanford, New...",3,1,3,"[1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, ...",3,3,36,33,6.861328
2,Michigan,2,"[Western Michigan, Washington, Northern Illino...","[Western Michigan, Washington, Northern Illino...",13,1,53,"[1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, ...",4,7,36,34,10.453125
3,UCLA,3,"[Hawaii, LSU, Stanford]","[Hawaii, LSU, Stanford, New Mexico State, Cent...",6,1,11,"[1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, ...",3,5,33,32,7.722656
4,Georgia,4,"[Clemson, UAB, South Carolina, Vanderbilt]","[Clemson, UAB, South Carolina, Vanderbilt, Geo...",13,1,7,"[1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, ...",4,5,26,22,8.031250
5,Oklahoma State,5,"[Tulsa, Boise State, Kansas State]","[Tulsa, Boise State, Kansas State, Arkansas St...",15,1,11,"[1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, ...",3,5,23,22,7.968750
6,UTSA,6,"[Illinois, Middle Tennessee, Memphis]","[Illinois, Middle Tennessee, Memphis, Nebraska...",7,1,3,"[1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 6, ...",3,3,22,24,5.891602
7,LSU,6,"[Central Michigan, Mississippi State]","[Central Michigan, Mississippi State, FIU, Lou...",9,1,3,"[1, 1, 2, 2, 2, 3, 3, 3, 4, 5, 6, 6, 7, 7, 8, ...",2,3,22,20,4.570312
8,Memphis,7,"[Arkansas State, Mississippi State]","[Arkansas State, Mississippi State, Louisiana ...",9,1,2,"[1, 1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 7, 7, 8, 8, ...",2,2,21,19,4.070312
9,Notre Dame,8,"[Florida State, Toledo, Purdue, Wisconsin]","[Florida State, Toledo, Purdue, Wisconsin, Bal...",16,1,14,"[1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, ...",4,5,20,16,8.437500


In [ ]:
input_year = 2021
if 1941 <= input_year <= 1945:
    input_year = 1940
url="https://en.wikipedia.org/wiki/NCAA_Division_I_Football_Bowl_Subdivision_alignment_history"
# use urllib and BeautifulSoup in order to open and copy the html code of the webpage to another variable
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'lxml')
# use soup.find in order to pull out only the html code related to the data wanted
team_list = soup.findAll('div')
A = []
for row in soup.findAll('li'):
    strip = row.text.strip()
    A.append(strip)
team_list
A = A[26:-50]

B = []
C = []
D = []
stop = "1"
for s in A:
    index = s.find(" 1")
    index2 = s.find(" 2")
    if index2 == -1 or index < index2 and index != -1:
        B.append(s[:index])
    else:
        B.append(s[:index2])

df = pd.DataFrame(A,columns = ['Team1'])
df['Team'] = B

df.replace("as West Texas State","", inplace=True, regex = True)
df.replace(" \(\)","", inplace=True, regex = True)
df.replace("present","2021", inplace=True, regex = True) # update each year
df.replace("future","2024", inplace=True, regex = True) # update each year
df.replace("1981","AAAA", inplace = True, regex=True)
df.replace("81","1981", inplace = True, regex=True)
df.replace("AAAA","1981", inplace = True, regex=True)
df.replace("1940","AAAA", inplace = True, regex=True)
df.replace("40","1940", inplace = True, regex=True)
df.replace("AAAA","1940", inplace = True, regex=True)
df.replace("-40","-1981", inplace = True, regex=True)
df.replace("Southern California","USC", inplace = True, regex=True)


for s, s2 in zip(df['Team1'], df['Team']):
    s3 = s.replace(s2,"")
    C.append(s3)
print(C)
np.linspace(1940,1946,7)
for s in C:
    print(s)
    a = []
    length = len(s)
    char = 1
    while char < length:
        if length < 6:
            a.append(int(s[char:char+4]))
            break
        if s == " 2022–2021[a]":
            print('t')
            a.append(2022)
            break
        if s[char+4] == ',':
            a.append(int(s[char:char+4]))
            char = char + 6
        else:
            year1 = int(s[char:char+4])
            year2 = int(s[char+5:char+9])
            years = range(year1, year2+1)
            for year in years:
                a.append(year)
            char = char + 11
    D.append(a)
        
df['Years'] = D
df.drop(['Team1'], axis = 1, inplace = True)
df
Team_List = []
if input_year >= 1937:
    for team, years in zip(df['Team'], df['Years']):
        if input_year in years:
            Team_List.append(team)
if input_year < 1937:
    Team_List = df['Team'].to_list()
Team_List

['s', ' 1957–2021', ' 1987–2021', ' 1937–1940, 1946–2021', ' 1977', ' 1974–1981, 2014–2021', ' 1939–1940, 1946–2021', ' 1946–1948, 1950–1954, 1956–2021', ' 1937–1940, 1946–2021', ' 1975–1981, 1992–2021', ' 1937–1940, 1946–2021', ' 1937–1940, 1946–2021', ' 1975–1981, 1983–2021', ' 1937–1940, 1946–2021', ' 1996–2021', ' 1939–1940, 1946–2021', ' 1939–1940, 1947–1965', ' 1961–1981, 1983–2021', ' 1937–1940, 1946–1981', ' 1938–1940, 1946–1947', ' 1962–1970, 1999–2021', ' 1938–1940, 1946–2021', ' 1937–1940, 1946–2021', ' 1976–1992', ' 1937–1940', ' 1948', ' 1938–1947', ' 1975–2021', ' 2015–2021', ' 1946–1948, 1950, 1977–1981', ' 1937–1939', ' 1946, 1948–1981, 1983–2021', ' 1939, 1946–1952, 1959–1981', ' 1937–1940, 1946–2021', ' 2017–2021', ' 1937–1940, 1946–1981', ' 1937–1940, 1946–2021', ' 1946–1947', ' 1940, 1946–2021', ' 1938–1940, 1946–1981', ' 1937–1940, 1946–1981', ' 1939–1942', ' 1937–1940, 1946–1981', ' 1937–1940, 1946–1953, 1967–1976', ' 1956–1976', ' 1940–1960', ' 1939–1964', ' 1939

['Air Force',
 'Akron',
 'Alabama',
 'Appalachian State',
 'Arizona',
 'Arizona State',
 'Arkansas',
 'Arkansas State',
 'Army',
 'Auburn',
 'Ball State',
 'Baylor',
 'Boise State',
 'Boston College',
 'Bowling Green',
 'Buffalo',
 'BYU',
 'California',
 'Central Michigan',
 'Charlotte',
 'Cincinnati',
 'Clemson',
 'Coastal Carolina',
 'Colorado',
 'Colorado State',
 'Duke',
 'East Carolina',
 'Eastern Michigan',
 'FIU',
 'Florida',
 'Florida Atlantic',
 'Florida State',
 'Fresno State',
 'Georgia',
 'Georgia Southern',
 'Georgia State',
 'Georgia Tech',
 'Hawaii',
 'Houston',
 'Illinois',
 'Indiana',
 'Iowa',
 'Iowa State',
 'Kansas',
 'Kansas State',
 'Kent State',
 'Kentucky',
 'Liberty',
 'Louisiana',
 'Louisiana–Monroe',
 'Louisiana Tech',
 'Louisville',
 'LSU',
 'Marshall',
 'Maryland',
 'Massachusetts',
 'Memphis',
 'Miami (FL)',
 'Miami (OH)',
 'Michigan',
 'Michigan State',
 'Middle Tennessee',
 'Minnesota',
 'Mississippi State',
 'Missouri',
 'Navy',
 'Nebraska',
 'Nevada',
 